___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# 배경 교체

### 녹색 배경으로 촬영된 영상

1. 배경으로 사용할 이미지를 읽어들여서 동영상과 같은 해상도로 [cv::resize()](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#ga47a974309e9102f5f08231edc7e7529d)
2. [cv::inRange()](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981)를 이용해서 mask 만들기
3. [옵션] ```GaussianBlur()```를 이용해서 mask를 더 부드럽게 만들기
4. 알파 블랜딩과 비슷하게 ```mask```를 이용해서 동영상과 배경 합성

```
mask = cv2.inRange(frame, (15, 142, 88), (55, 215, 150))
```

frame의 필셀들 중에서 $15 \leq B \leq 55$, $142 \leq G \leq 215$, $88 \leq R \leq 150$를 모두 만족하는 픽셀의 위치에 해당하는 mask는 255, 그 외에는 0



```
mask = cv2.GaussianBlur(mask, (15, 15), 0)
```


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("green_background.mp4")

if not cap.isOpened():
    print("Error opening video  file")

fps = cap.get(cv2.CAP_PROP_FPS)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

background = cv2.imread("point_fermin_park.jpg")
background = cv2.resize(background, (int(width), int(height)))
background = cv2.GaussianBlur(background, (25, 25), 0)

fourcc = cv2.VideoWriter_fourcc(*"X264")
out = cv2.VideoWriter("without_gaussian.mp4", fourcc, fps, (int(width), int(height)))

while cap.isOpened():

    ret, frame = cap.read()

    if not ret:
        print("End of frames.")
        break

    # 녹색과 비슷하면 255, 아니면 0
    mask = cv2.inRange(frame, (15, 142, 88), (55, 215, 150))
    mask = cv2.GaussianBlur(mask, (15, 15), 0)
    # cv2.imwrite("mask_blur.jpg", mask)
    # cv2.waitKey(0)
    # break
    mask = np.expand_dims(mask, 2) / 255.0

    comp = frame * (1.0 - mask) + background * mask
    comp = comp.astype(np.uint8)

    out.write(comp)
    cv2.imshow("Comp", comp)

    if cv2.waitKey(int(100.0 / fps)) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

### 임의의 배경 교체 (딥러닝)

[MediaPipe](https://google.github.io/mediapipe/)의 [Selfie Segmentation](https://google.github.io/mediapipe/solutions/selfie_segmentation#mediapipe-selfie-segmentation) 사용

```
pip install mediapipe
```



In [ ]:
# 기초적인 사용 방법
import cv2
import mediapipe as mp
import numpy as np

with mp.solutions.selfie_segmentation.SelfieSegmentation(
    model_selection=1
) as selfie_segmentation:

    image = cv2.imread("arbitrary_background.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # for neural network

    # 백그라운드가 1.0이 되도록 수정
    mask = 1.0 - selfie_segmentation.process(image).segmentation_mask

cv2.imwrite("mediapipe_mask.jpg", mask * 255.0)
# cv2.imshow("Mask", mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()